In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Added modules

In [ ]:
def get_theta(x, mu, sigma):
    """
    x: fraction of similar neighbours
    mu: optimal fraction of similar neighbours
    sigma: acceptance range
    """
    theta = np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))
    return theta

In [ ]:
def gaussian_function(x, mu, sigma):
    return np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

# Example usage:
x_values = np.linspace(0, 1, 100)
mu = 0.5  # Peak in the middle
sigma = 0.6  # Controls the width

y_values = gaussian_function(x_values, mu, sigma)

# You can plot the function to visualize it
import matplotlib.pyplot as plt

plt.plot(x_values, y_values)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title(f'Gaussian Function with Peak at {mu}')
plt.show()

# Sobol Sensitivity Analysis

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol

In [ ]:
import modules as modules
import model as model

# # Create and run the model
model = model.Schelling(
     property_value_func=modules.property_value_quadrants,
     income_func=modules.income_func,
     desirability_func=modules.desirability_func,
     utility_func=modules.utility_func,
     price_func=modules.price_func,
     compute_similar_neighbours=modules.compute_similar_neighbours,
     height=20,
     width=20,
     radius=1,
     density=0.8,
     minority_pc=0.2,
     alpha=0.5,
     seed=42
 )

In [ ]:
# # Run the model for a certain number of steps
for i in range(5):
     #print(i)
     #print(model.entropy)
     model.step()

# Exemplory use of accessing datacollector
model_data = model.datacollector.get_model_vars_dataframe()
agent_data = model.datacollector.get_agent_vars_dataframe()
print(model_data)
print(agent_data)

In [ ]:
# Step 1: Problem definition
problem = {
    'num_vars': 6,
    'names': ['density', 'minority_pc', 'property_value_weight', 'alpha', 'mu_theta', 'sigma_theta'],
    'bounds': [[0,1], [0,1], [0,1], [0,1], [0,1], [0,1]]
}

In [ ]:
# Step 2: Generate the samples (power of 2 for best performance)
samples = saltelli.sample(problem, 1) #2**5) #2024

In [ ]:
# Step 3: Run the model for generated samples
import model
import modules

# First define the model such that it runs for a specified number of time steps in a function 
def run_schelling_model(property_value_func,
                        income_func,
                        desirability_func,
                        utility_func,
                        price_func,
                        compute_similar_neighbours,
                        height,
                        width,
                        radius,
                        params,
                        #density,
                        #minority_pc,
                        #alpha,
                        #property_value_weight,
                        #mu_theta,
                        #sigma_theta,
                        seed,
                        num_steps):
    # Initialize the model
    model_instance = model.Schelling(
        property_value_func=property_value_func,
        income_func=income_func,
        desirability_func=desirability_func,
        utility_func=utility_func,
        price_func=price_func,
        compute_similar_neighbours=compute_similar_neighbours,
        height=height,
        width=width,
        radius=radius,
        density=params[0], #density,
        minority_pc=params[1], #minority_pc,
        alpha=params[2], #alpha,
        income_scale=1.5, # the scale by which the income is higher than the property value
        property_value_weight=params[3], #property_value_weight,
        mu_theta=params[4], #mu_theta,
        sigma_theta=params[5],
        seed=seed
    )

    # Run the model for the specified number of steps and collect entropy values
    entropies = []
    for _ in range(num_steps):
        model_instance.step()
        entropies.append(model_instance.entropy)

    # Compute mean and standard deviation of entropy
    entropy_mean = np.mean(entropies)
    entropy_std = np.std(entropies)

    # 
    model_data = model_instance.datacollector.get_model_vars_dataframe()
    agent_data = model_instance.datacollector.get_agent_vars_dataframe()

    # Return the model instance and entropy values
    return model_instance, np.array(entropies), entropy_mean, entropy_std, model_data, agent_data

# Exemplory use
"""# Example of calling the function with specific parameters
model_result = run_schelling_model(
    property_value_func=modules.property_value_quadrants,
    income_func=modules.income_func,
    desirability_func=modules.desirability_func,
    utility_func=modules.utility_func,
    price_func=modules.price_func,
    compute_similar_neighbours=modules.compute_similar_neighbours,
    height=20,
    width=20,
    radius=1,
    density=0.8,
    minority_pc=0.2,
    alpha=0.5,
    seed=42,
    num_steps=5
)"""

In [ ]:
samples[0]

In [ ]:
# Next compute the model for the different parameter settings 
Y = run_schelling_model(property_value_func=modules.property_value_quadrants,
    income_func=modules.income_func,
    desirability_func=modules.desirability_func,
    utility_func=modules.utility_func,
    price_func=modules.price_func,
    compute_similar_neighbours=modules.compute_similar_neighbours,
    height=20,
    width=20,
    radius=1,
    params=samples[0],
    seed=42,
    num_steps=2)

In [ ]:
Y[3]

In [ ]:
import numpy as np
from SALib.sample import saltelli
from SALib.analyze import sobol

# Step 1: Define the problem
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[0, 1], [0, 1], [0, 1]]
}

# Step 2: Generate samples
param_values = saltelli.sample(problem, 1000)

# Step 3: Evaluate the model
# Define your model (this is an example model)
def model(X):
    x1 = X[:, 0]
    x2 = X[:, 1]
    x3 = X[:, 2]
    return x1 + x2 + x3

# Run the model for generated samples
Y = model(param_values)

In [ ]:
print(Y)

In [ ]:
problem

In [ ]:
# Step 4: Perform Sobol sensitivity analysis
sobol_indices = sobol.analyze(problem, Y, print_to_console=True)

# Step 5: Extract results
Si = sobol_indices

# Print the first-order, second-order, and total-order sensitivity indices
print("First-order indices:", Si['S1'])
print("Second-order indices:", Si['S2'])
print("Total-order indices:", Si['ST'])